<h1>
<center>
Final Exam
</center>
</h1>
<div class=h1_cell>
<p>
The final has 3 basic parts. Part 1 is wrangling a dataset and using ann_flex to train and test a model (40 points). Part 2 is to look at the results of testing in part 1 and do some filtering on the test samples (30 points). Part 3 is an exploration of an algorithm that tries to reduce weight updates (30 points). It uses a simulation loop.
 <p>
   I will give you the dataset to use. You have to supply your library.

In [0]:
!rm library_w19_deep_2.py

In [2]:
from google.colab import files
files.upload()

Saving library_w19_deep_2.py to library_w19_deep_2.py


{'library_w19_deep_2.py': b"import numpy as np\r\n\r\ndef sigmoid(x):  \r\n    return 1/(1+np.exp(-x))\r\n\r\ndef mse(z,y):\r\n  return (z-y)**2\r\n\r\ndef mse_der(z,y):\r\n  return z-y\r\n\r\ndef sigmoid_der(x):  \r\n    return sigmoid(x)*(1-sigmoid(x))\r\n\r\ndef ann_simple(all_samples, labels, weights, bias, hypers={}):\r\n  \r\n  '''\r\n  Can build an ANN with n input nodes and one output node.\r\n  Uses sigmoid and mse.\r\n  '''\r\n  \r\n  input_n = all_samples.shape[1]  #number of inputs in each sample\r\n  \r\n  assert weights.shape == (input_n,1), 'weights needs to have same shape as sample'\r\n  assert all_samples.shape[0] >= 1, 'all_samples must represent 1 or more samples'\r\n  assert bias.shape == (1,) , 'a single bias weight for output node'\r\n  assert labels.shape[1] == 1, 'actual value for the 1 output node'\r\n  assert labels.shape[0] == all_samples.shape[0], 'labels must match up with samples'\r\n  \r\n  hyper_keys = [*hypers]  #fails on 2.7\r\n  target_set = set(['ep

In [3]:
from library_w19_deep_2 import *

%who function

ann_flex	 ann_flex_predictor	 ann_flex_tester	 ann_predictor	 ann_simple	 ann_simple_batch	 ann_tester	 from_scratch	 from_scratch_batch	 
from_scratch_flex	 mse	 mse_der	 sigmoid	 sigmoid_der	 


In [0]:
import numpy as np
import pandas as pd

In [0]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTST3CURFkC0lHSfR9mGzBAvCmLWl06kw9NSBC3UuD9hA1SlPjh807BRxfnvhq5hrFyRz5ZCTGrs7fr/pub?output=csv'
movie_table = pd.read_csv(url)

In [6]:
movie_table = movie_table[:200]  #trim it down to 200 rows
len(movie_table)

200

In [7]:
movie_table.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


<h2>Question 1: get your wrangling boots on (15 points)</h2>
<p>
I am going to tell you what I want and then let you figure out the steps to get there.
  <p>
   1. I wanti `mdb_score` to be binned into 3 values using `qcut`. I then want that one-hot encoded into the list of labels for `ann_flex`. FInal shape of your `labels` is (n,3).
   2. I would like to use the following columns as part of a sample: `facenumber_in_poster,	num_user_for_reviews,	budget`. Please normalize these columns.
   3. I would like to use  `language` as also part of a sample.
   4. You should end up with 4 values in each sample.
 <br><p> 
Your `ann_flex` function will choke on `NaN` values so make sure you remove rows that contain them.
   <p>
When you have `feature_set` (the list of samples) and `labels` ready to go, I'll meet you at the next question.
  


In [0]:
movie_table = movie_table.dropna()
labels = ['Low', 'Med', 'High']
movie_table['imdb_binned'] = pd.qcut(movie_table['imdb_score'], 3, labels=labels)
one_hot_imdb = pd.get_dummies(movie_table['imdb_binned'],prefix='imdb',dummy_na=False)  #no empties so use False
movie_table = movie_table.join(one_hot_imdb)
movie_table['fip_normed'] = movie_table['facenumber_in_poster']/movie_table['facenumber_in_poster'].max().astype(np.float64)
movie_table['nufr_normed'] = movie_table['num_user_for_reviews']/movie_table['num_user_for_reviews'].max().astype(np.float64)
movie_table['bud_normed'] = movie_table['budget']/movie_table['budget'].max().astype(np.float64)

In [25]:
raw_table = movie_table.copy()
ann_table = movie_table[['fip_normed', 'nufr_normed', 'bud_normed', 'language']].copy()
ann_table['language'] = ann_table.apply(lambda row: 1 if row['language'] == 'English' else 0, axis=1)  
test_labels = movie_table[['imdb_Low', 'imdb_Med', 'imdb_High']].copy()
ann_table.head(1)

,fip_normed,nufr_normed,bud_normed,language
0,0.0,0.654382,0.79,1


In [26]:
feature_set = ann_table.values.tolist()
feature_set = np.array(feature_set)
print(feature_set.shape)
feature_set[:5]

(193, 4)


array([[0.        , 0.65438183, 0.79      , 1.        ],
       [0.        , 0.26526677, 1.        , 1.        ],
       [0.125     , 0.21298479, 0.81666667, 1.        ],
       [0.        , 0.57874438, 0.83333333, 1.        ],
       [0.125     , 0.15813156, 0.879     , 1.        ]])

My feature set has this shape and first 5 values:
<pre>
(194, 4)
array([[0.        , 0.64881341, 0.78894472, 1.        ],
       [0.        , 0.25342913, 1.        , 1.        ],
       [0.125     , 0.20030481, 0.81574539, 1.        ],
       [0.        , 0.57195733, 0.83249581, 1.        ],
       [0.125     , 0.14456782, 0.87839196, 1.        ]])
       </pre>

In [16]:
labels = np.array(test_labels)
print(labels.shape)
labels[0:5]

(193, 3)


array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0]], dtype=uint8)

My labels has this shape and first 5 values:
<pre>
(194, 3)
array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0]], dtype=uint8)
</pre>

<h2>Question 2: train a model (15 points)</h2>

Here's where you earn your 15 takehome points. Show that your `ann_flex` works correctly.
<p>
Train your model with the first 150 rows. Use `epochs` at 3000 and `cost-reporting` at whatever you like. Everything else can be default values. Here are my resulting `weights` and `biases`.
<pre>
array([[ 0.49499155, -1.51876998,  1.2508087 ],
       [-7.17156319, -0.38214105,  3.18578869],
       [-2.76749659, -1.26703916,  3.34669796],
       [ 0.62092268,  0.17293429, -2.23830144]])
array([ 1.41175081,  0.4811565 , -2.15045844])
</pre>

In [31]:
(weights, biases) = from_scratch_flex(feature_set[:150], labels[:150], hypers={'epochs':3000, 'cost-reporting': 1000})

(0, 0.711989156200089)
(1000, 0.6819104060697414)
(2000, 0.681835817985708)
(3000, array([0.22624216, 0.23840047, 0.21719318]))


In [32]:
weights, biases

(array([[ 0.49618789, -1.52004977,  1.25204887],
        [-7.30285707, -0.39072486,  3.23561355],
        [-2.78507374, -1.2693359 ,  3.36553471],
        [ 0.68756875,  0.17857116, -2.27384152]]),
 array([ 1.47839688,  0.48679337, -2.18599852]))

<h2>Question 3: test your model (10 points)</h2>

You used the first 150 samples to train your model. Test your model with remaining samples from 200 you started with.
<p>
  For computing accuracy, you will have 3 raw values as output. Use the max value as the position of the 1. For instance, if you had output
  <p>
    `       array([0.55081372, 0.36331553, 0.11846781])`
    <p>
      you would predict '(1,0,0)`. The max value is in the 0th position so would place the 1 there. Once you have done this conversion, you can compare your predictions against labels to get accuracy.
<p>
        My accuracy is `0.36`. Just barely above the roll of a 3-sided dice.
  <p>
    One last note. You will need the raw output of your model in the next question so good to keep track of it here so you have it there.


In [33]:
(weights, biases) = from_scratch_flex(feature_set, labels, hypers={'epochs':3000, 'cost-reporting': 1000})

(0, 0.7128927069257365)
(1000, 0.680402403773903)
(2000, 0.6802879669670459)
(3000, array([0.23361516, 0.22474954, 0.22192326]))


In [53]:
raw_table['imdb_dummy'] = raw_table['imdb_binned']
raw_table['imdb_dummy'].replace('High', 2, inplace=True)
raw_table['imdb_dummy'].replace('Med', 1, inplace=True)
raw_table['imdb_dummy'].replace('Low', 0, inplace=True)
raw_table.head(1)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,aspect_ratio,movie_facebook_likes,imdb_binned,imdb_Low,imdb_Med,imdb_High,fip_normed,nufr_normed,bud_normed,imdb_dummy
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,1.78,33000,High,0,0,1,0.0,0.654382,0.79,2


In [54]:
test_table_simp = raw_table.filter(['imdb_dummy'])
test_list_simp = test_table_simp['imdb_dummy'].values.tolist()
print(test_list_simp[:10])

[2, 1, 1, 2, 1, 0, 2, 2, 2, 1]


In [58]:
predictions = ann_flex_tester(feature_set, weights, biases)
preds_values = [np.argmax(p) for p in predictions]
zipped = list(zip(preds_values, test_list_simp))
sum([1 if p==a else 0 for p,a in zipped])/len(zipped)

0.45077720207253885

<h2>Question 4:  filtering the test set (30 points)</h2>
  
I'd like to reduce the number of samples in the feature set I use for testing. I know we are only using a small number of test samples to keep processing costs down in the exam, but normally the size could be in the millions. What I would like to do is find the samples that have similar errors when doing prediction. Here is what I propose to do: look at pairwise sets of samples. If the 2 samples in the pair have roughly the same overall error (i.e., within epsilon), I can remove one of them from the test set: it is redundant.
  <p>
  I'm going to ask you to help me with the first part of this problem. I'd like an ordered list, pairwise, of all the samples along with the absolute difference in their overall errors. That's a mouthful. Let me give you some more details. I am using results we got from question 3.
  
  1. As you know, `labels` is a list of  triples where each triple is one-hot encoded.  Your raw output is also a triple of  values. What do I mean by "raw output"? I mean the actual 3 values that you get from your model, e.g., `array([0.55081372, 0.36331553, 0.11846781])`. For each such raw output, calculate the absolute difference between the 2 triples, i.e., your raw output triple and the label triple. This will give you a new triple of errors. So if the label for my example raw output above was `(0,0,1)`, I would get this as the error triple: `(abs(0-0.55081372), abs(0-0.36331553), abs(1-0.11846781))`. 
  2. Sum the error triple. That sum is what I want to store with the sample, i.e., I will record the sample index (or equivalently the label index) and a single summed error that goes with it. Note that this is different than the practice problem where you were doing an average. We are not doing any averaging.
  3. I'd now like you to go pairwise through the samples and for each unique pair, record the absolute difference between their summed errors. As an example, if you were using a dictionary to keep track of your results, you might see an entry `(0,1): .003`, which signifies that the absolute difference between sample 0 and sample 1 is .003. What does that actually mean? The two samples are fairly close in their errors. Both errors could be quite high. But the absolute difference between them is small.
  4. Order the list based on ascending absolute differences.

  You can stop there. Normally I would then use your ordered list to filter out samples that have errors within some epsilon of each other. We do not need 2 separate samples that mimic each other in terms of errors they produce. We can just keep one of them and toss the other.
<p>
  You can see my final results below for the first 10 in my ordered list. To keep processing times down, I trimmed predictions and labels to [:15], i.e. the first 15.
  <pre>
  [((0, 3), 0.002331643103141934),
 ((7, 10), 0.01036188464574006),
 ((11, 12), 0.016973839107757627),
 ((0, 2), 0.028251848582276695),
 ((2, 3), 0.03058349168541863),
 ((5, 14), 0.0453548688750367),
 ((10, 14), 0.04570192275791962),
 ((6, 8), 0.05405101534935941),
 ((7, 14), 0.05606380740365968),
 ((5, 6), 0.06109838820689739)]
 </pre>
 <p>
  What this means is that the total error for sample 0 and the total error for sample 3 are within 0.00233 of each other using absolute difference.

To keep processing time within reason, only work with the first 15 predictions from your testing. Here are mine. If your first 15 don't match my first 15, go ahead and use mine.

In [0]:
def calc_errors(raw, labels):
  errors = []
  errors2 = []
  for i in range(len(labels)):
    errors.append((abs(labels[i][0] - raw[i][0]), abs(labels[i][1] - raw[i][1]), abs(labels[i][2] - raw[i][2])))
  for item in range(len(errors)):
    errors2.append(np.sum(errors[item]))
  return errors2
    
errors = calc_errors(full_raw, labels)


In [78]:
predictions_15 = [([0.55081372, 0.36331553, 0.11846781]),
 ([0.63322896, 0.5091635 , 0.06235565]),
 ([0.7097191 , 0.50770786, 0.04740651]),
 ([0.37215582, 0.45147866, 0.11034364]),
 ([0.45966018, 0.50442377, 0.08079772]),
 ([0.15938415, 0.47507109, 0.15527721]),
 ([0.09379516, 0.47731639, 0.18199096]),
 ([0.64372884, 0.28360809, 0.14045216]),
 ([0.66493551, 0.32965622, 0.1117983 ]),
 ([0.57558268, 0.5153202 , 0.06451307]),
 ([0.28899897, 0.45579208, 0.12019616]),
 ([0.63175644, 0.41794559, 0.08506059]),
 ([0.20354411, 0.49121776, 0.12801538]),
 ([0.70649204, 0.25956437, 0.13452677]),
 ([0.60337277, 0.37912699, 0.09892885])]
predictions_15

[[0.55081372, 0.36331553, 0.11846781],
 [0.63322896, 0.5091635, 0.06235565],
 [0.7097191, 0.50770786, 0.04740651],
 [0.37215582, 0.45147866, 0.11034364],
 [0.45966018, 0.50442377, 0.08079772],
 [0.15938415, 0.47507109, 0.15527721],
 [0.09379516, 0.47731639, 0.18199096],
 [0.64372884, 0.28360809, 0.14045216],
 [0.66493551, 0.32965622, 0.1117983],
 [0.57558268, 0.5153202, 0.06451307],
 [0.28899897, 0.45579208, 0.12019616],
 [0.63175644, 0.41794559, 0.08506059],
 [0.20354411, 0.49121776, 0.12801538],
 [0.70649204, 0.25956437, 0.13452677],
 [0.60337277, 0.37912699, 0.09892885]]

Similarily, we need the first 15 labels from the test set. Here are mine. If your first 15 don't match my first 15, go ahead and use mine.

In [81]:
labels_15 = labels[:14]
labels_15

array([[0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0]], dtype=uint8)

In [0]:
labels_15

array([[0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=uint8)

In [86]:
vals = []
for i in range(len(labels_15)):
  vals.append(calc_errors(labels_15[i], predictions_15[i]))
  
vals

TypeError: ignored

<h2>Question 5: ignoring weight changes (30 points)</h2>

For efficiency reasons, we may want to avoid making weight changes unless the change really matters. I'd like you to use a simulation loop to test out this algorithm:

 1. Keep track of the weight changes from a previous sample. Unlike momentum, you need only to keep track of a single change matrix. I call this my history cache. Initially it will start with a zeroed out change matrix.
 2. Compare the current weight change (simulated) with the weight change matrix in the history cache.
 3. If they are close, do nothing; go immediately to the next sample. You can just ignore the weight change for the current sample and do nothing to the history cache.
 4. If they are not close, then go ahead and make the (simulated) weight change to weights and store the current (simulated) weight change in your history cache, replacing what ever was there.
<p>
I hope you are asking what the heck does "close" mean. Well, I'll tell you. Those clever folks at numpy headquarters have defined a method to compare the closeness of 2 matrices. Here is what I would like you to use:
<pre>
  np.allclose(history_cache, simulated_changes, atol=.09)
</pre>
Pretty cool, huh. Normally `atol`, the tolerance the function uses, would be a hyper-parameter. I am just going to use .09.
<p>
I printed out some results for you to compare against. I printed out the starting weights and the ending weights. And every time I ignored a weight change I printed the sample index and the string `close`. Here is what I got. You can see I avoided 4 updates to weights.
<pre>
[[-0.02509198  0.09014286]
 [ 0.04639879  0.0197317 ]
 [-0.06879627 -0.0688011 ]]
==============================
(1, 'close')
(2, 'close')
(5, 'close')
(9, 'close')
array([[ 0.11476858, -0.0774422 ],
       [ 0.20985929, -0.14123441],
       [-0.04380995, -0.18179776]])
</pre>

In [83]:
ar = np.zeros([3, 2])
ar

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [0]:
input_n = 3    #how many input nodes
output_n = 2   #how many output nodes
sample_n = 10  #how many simulated samples to try

np.random.seed(42)
weights = .2*np.random.rand(input_n,output_n) - .1 #initial value of weights matrix
history_cache = np.zeros([3, 2])

print(weights)
print('='*30)

for i in range(sample_n):
  #generate simulated weight changes as if coming from backprop
  simulated_changes = .2*np.random.rand(input_n,output_n) - .1
  past1 = history_cache.copy()
  weights = weights + (-simulated_changes- (.09*past5))
  history_cache = simulated_changes



weights  #values when done with for loop